In [2]:
import importlib
import pandas as pd

pd.options.display.max_colwidth = 10
pd.options.display.max_rows = 4

In [3]:
file_path = './data/processed/dataset.pkl'
df = pd.read_pickle(file_path, compression='zstd')
# with open('./data/processed/dataset.pkl', 'rb') as f:
#     df = joblib.load(f)
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,94,92.693...,PANATL...,DF Con...,1,2011-12-31,NaT,2010-12-31,-1,1,Ativo ...,True,NaN,2.8259...
1,94,92.693...,PANATL...,DF Con...,1,2011-12-31,NaT,2010-12-31,-1,1.01,Ativo ...,True,NaN,2.1937...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17674197,518700,70.216...,BRAMEX...,DF Ind...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.0...,PNB,False,NaN,2.1489...
17674198,518700,70.216...,BRAMEX...,DF Ind...,1,2013-12-31,2013-01-01,2013-12-31,0,3.99.02,Lucro ...,False,NaN,0.0000...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17674199 entries, 0 to 17674198
Data columns (total 14 columns):
 #   Column         Dtype   
---  ------         -----   
 0   CD_CVM         category
 1   CNPJ_CIA       category
 2   DENOM_CIA      category
 3   GRUPO_DFP      category
 4   VERSAO         category
 5   DT_REFER       category
 6   DT_INI_EXERC   category
 7   DT_FIM_EXERC   category
 8   ORDEM_EXERC    category
 9   CD_CONTA       category
 10  DS_CONTA       category
 11  ST_CONTA_FIXA  category
 12  COLUNA_DF      category
 13  VL_CONTA       float64 
dtypes: category(13), float64(1)
memory usage: 491.7 MB


In [5]:
df.query("CD_CVM == 9512")

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
3144784,9512,33.000...,PETROL...,DF Con...,1,2012-12-31,NaT,2011-12-31,-1,1,Ativo ...,True,NaN,6.0009...
3144785,9512,33.000...,PETROL...,DF Con...,1,2012-12-31,NaT,2011-12-31,-1,1.01,Ativo ...,True,NaN,1.2116...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181896,9512,33.000...,PETROL...,DF Ind...,3,2010-12-31,2010-01-01,2010-12-31,0,3.99.0...,PN,False,NaN,3.5500...
3181897,9512,33.000...,PETROL...,DF Ind...,3,2010-12-31,2010-01-01,2010-12-31,0,3.99.02,Lucro ...,False,NaN,0.0000...


In [48]:
class Company():
    def __init__(self, cvm_number: int):
        self.cvm_number = int(cvm_number)
    def assets(self):
        df = pd.read_pickle(file_path, compression='zstd')
        df.query("CD_CVM == @self.cvm_number", inplace=True)
        df['CD_CONTA_L1'] = df['CD_CONTA'].str[0].astype(int)
        df.query("CD_CONTA_L1 == 1", inplace=True)
        df.sort_values(['DT_REFER', 'ORDEM_EXERC'], inplace=True)
        df.reset_index(drop=True, inplace=True)
        return df

In [49]:
petro = Company(9512)
petro.assets()

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA,CD_CONTA_L1
0,9512,33.000...,PETROL...,DF Con...,3,2010-12-31,NaT,2009-12-31,-1,1,Ativo ...,True,NaN,3.5041...,1
1,9512,33.000...,PETROL...,DF Con...,3,2010-12-31,NaT,2009-12-31,-1,1.01,Ativo ...,True,NaN,7.4373...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,9512,33.000...,PETROL...,DF Ind...,1,2021-09-30,NaT,2021-09-30,0,1.02.0...,Intang...,True,NaN,0.0000...,1
4623,9512,33.000...,PETROL...,DF Ind...,1,2021-09-30,NaT,2021-09-30,0,1.02.0...,Contra...,True,NaN,0.0000...,1


In [47]:
df['CD_CONTA'].str[0].astype(int)

0           1
1           1
           ..
17674197    3
17674198    3
Name: CD_CONTA, Length: 17674199, dtype: int64